In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

In [2]:
df = pd.read_csv("supply_chain_data.csv")
df.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [3]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [4]:
df.columns

Index(['product_type', 'sku', 'price', 'availability',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics',
       'stock_levels', 'lead_times', 'order_quantities', 'shipping_times',
       'shipping_carriers', 'shipping_costs', 'supplier_name', 'location',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   product_type             100 non-null    object 
 1   sku                      100 non-null    object 
 2   price                    100 non-null    float64
 3   availability             100 non-null    int64  
 4   number_of_products_sold  100 non-null    int64  
 5   revenue_generated        100 non-null    float64
 6   customer_demographics    100 non-null    object 
 7   stock_levels             100 non-null    int64  
 8   lead_times               100 non-null    int64  
 9   order_quantities         100 non-null    int64  
 10  shipping_times           100 non-null    int64  
 11  shipping_carriers        100 non-null    object 
 12  shipping_costs           100 non-null    float64
 13  supplier_name            100 non-null    object 
 14  location                 10

In [6]:
df.isna().sum()

product_type               0
sku                        0
price                      0
availability               0
number_of_products_sold    0
revenue_generated          0
customer_demographics      0
stock_levels               0
lead_times                 0
order_quantities           0
shipping_times             0
shipping_carriers          0
shipping_costs             0
supplier_name              0
location                   0
lead_time                  0
production_volumes         0
manufacturing_lead_time    0
manufacturing_costs        0
inspection_results         0
defect_rates               0
transportation_modes       0
routes                     0
costs                      0
dtype: int64

In [7]:
df.replace(" ", np.nan, inplace=True)
df.drop_duplicates(inplace=True)

In [8]:
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].astype(str)

In [9]:
df.head()

,product_type,sku,price,availability,number_of_products_sold,revenue_generated,customer_demographics,stock_levels,lead_times,order_quantities,...,location,lead_time,production_volumes,manufacturing_lead_time,manufacturing_costs,inspection_results,defect_rates,transportation_modes,routes,costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


### Feature Engineering

In [10]:
# Total Lead Time
df["total_lead_time"] = df["lead_times"] + df["manufacturing_lead_time"]

In [11]:
# Logistic Cost per Unit
df["cost_per_unit"] = df["costs"] / df["number_of_products_sold"]

In [12]:
# Shipping Days
df["shipping_delay_flag"] = np.where(df["shipping_times"] > df["lead_times"], 1, 0)

In [13]:
# Inventory Health Score
df["inventory_health"] = df["stock_levels"] - df["order_quantities"]

In [14]:
# Defeat Category
df["defeat_category"] = pd.cut(df["defect_rates"],
                              bins=[0, 1, 3, 10],
                              labels=["Low Defects", "Medium Defects", "High Defects"])

In [15]:
# Profit estimate
df["estimated_profit"] = (df["price"] - df["manufacturing_costs"]) * df["number_of_products_sold"]

In [16]:
df.head()

,product_type,sku,price,availability,number_of_products_sold,revenue_generated,customer_demographics,stock_levels,lead_times,order_quantities,...,defect_rates,transportation_modes,routes,costs,total_lead_time,cost_per_unit,shipping_delay_flag,inventory_health,defeat_category,estimated_profit
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,0.226410,Road,Route B,187.752075,36,0.234105,0,-38,Low Defects,18869.557293
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,4.854068,Road,Route B,503.065579,60,0.683513,0,16,High Defects,-13817.108813
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,4.580593,Air,Route C,141.920282,37,17.740035,0,-87,High Defects,-154.946688
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,4.746649,Rail,Route A,254.776159,31,3.069592,0,-36,High Defects,2119.703934
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,3.145580,Air,Route A,923.440632,6,1.060207,1,-51,High Defects,-76003.167835


In [17]:
df.to_csv("supply_chain_clean_data.csv")